# SN-NODE

In [36]:
import time
import diffrax
import equinox as eqx
import jax
import jax.nn as jnn
import jax.numpy as jnp
import jax.random as jr
import jax.random as jrandom
import optax

import csv
import numpy as np
from scipy.interpolate import interp1d
import os
import random
import pandas as pd

In [38]:
class Func(eqx.Module):
    mlp: eqx.nn.MLP
    linear_weight: jnp.ndarray  # Trainable parameter 'a'

    def __init__(self, data_size, width_size, depth, *, key, **kwargs):
        super().__init__(**kwargs)
        self.mlp = eqx.nn.MLP(
            in_size=data_size,
            out_size=data_size,
            width_size=width_size,
            depth=depth,
            activation=jnn.sigmoid,
            key=key,
        )
        
        self.linear_weight = jrandom.normal(key, (data_size,))* 1e-1

    def __call__(self, t, y, args):
        
        # Linear term: a * X
        linear_term = self.linear_weight * y
        
        # Nonlinear term: MLP(X)
        nonlinear_term = self.mlp(y)

        # Return the sum: a * X + MLP(X)
        return linear_term + nonlinear_term


In [20]:
class NeuralODE(eqx.Module):
    func: Func

    def __init__(self, data_size, width_size, depth, *, key, **kwargs):
        super().__init__(**kwargs)
        self.func = Func(data_size, width_size, depth, key=key)

    def __call__(self, ts, y0):
        solution = diffrax.diffeqsolve(
            diffrax.ODETerm(self.func),
            diffrax.Tsit5(),
            t0=ts[0],
            t1=ts[-1],
            dt0=ts[1] - ts[0],
            y0= y0,
            stepsize_controller=diffrax.PIDController(rtol=1e-3, atol=1e-6),
            saveat=diffrax.SaveAt(ts=ts),
            
        )
        return solution.ys

In [21]:
def dataloader(arrays, batch_size, *, key):
    dataset_size = arrays[0].shape[0]
    assert all(array.shape[0] == dataset_size for array in arrays)
    indices = jnp.arange(dataset_size)
    while True:
        perm = jr.permutation(key, indices)
        (key,) = jr.split(key, 1)
        start = 0
        end = batch_size
        while end <= dataset_size:
            batch_perm = perm[start:end]
            yield tuple(array[batch_perm] for array in arrays)
            start = end
            end = start + batch_size

In [22]:
def dataload(number):
    file_path = r'xxx' # replace this with your file path
    column_data = [] 
    
    total_rows_to_collect = 24*number  # Total data size
   
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader) 
        for _, row in zip(range(total_rows_to_collect), reader):
            column1_data.append(row[0])  
            
    column_array = np.array(column_data)
    
    return column_array

In [23]:
def reshape_minmax(data, number):
    
    reshaped_data = data.reshape(number, 24)
    extended_data = np.zeros((number, 25))

    for i in range(number - 1):
        extended_data[i, :-1] = reshaped_data[i, :]
        extended_data[i, -1] = reshaped_data[i + 1, 0] 

    extended_data[-1, :-1] = reshaped_data[-1, :]
    extended_data[-1, -1] = reshaped_data[-1, -1]

    interpolated_data = np.zeros((number, 1000))

    x_original = np.linspace(0, 24, 25)
    
    x_new = np.linspace(0, 24, 1000)

    for i in range(number):

        f = interp1d(x_original, extended_data[i], kind='cubic')

        interpolated_data[i] = f(x_new)
        
    data_min = np.zeros(number)
    data_max = np.zeros(number)
    normalized_data = np.zeros((number, 1000))

    for i in range(number):
        data_min[i] = interpolated_data[i, :].min()
        data_max[i] = interpolated_data[i, :].max()
        normalized_data[i, :] = (interpolated_data[i, :] - data_min[i]) / (data_max[i] - data_min[i])
 
    return  normalized_data,data_min,data_max

In [24]:
def combination(data, number):
    
    ts = jnp.linspace(0, 24, 1000)
        
    ys_1 = data.reshape(number, 1000, 1)

    t_replicated = np.tile(ts, (number, 1)).reshape(number, 1000, 1)
    
    ys = np.concatenate([t_replicated, ys_1], axis=2)
    
    return ts,ys

In [25]:
def main(
    dataset_size=100,
    datasize_tr = 75,
    steps_strategy=(2000,9900),
    randomseed = 1,
    batch_size=10,
    lr_strategy=(3e-3,3e-3),
    length_strategy=(0.1,1),
    width_size=100,
    depth=5,
    seed=5678,
    plot=True,
    print_every=500,
):
    key = jr.PRNGKey(seed)
    data_key, model_key, loader_key = jr.split(key, 3)
    
    energy = dataload(dataset_size)
    [data,data_min,data_max] = reshape_minmax(energy, dataset_size)
    [ts,ys] = combination(data, dataset_size)
    
    random.seed(randomseed)  
    selected_indices = random.sample(range(dataset_size), datasize_tr)
    selected_array = ys[selected_indices, :, :]
    ys = selected_array
    
    data_max = [data_max[item - 1] for item in selected_indices]
    data_min = [data_min[item - 1] for item in selected_indices]
                
    _, length_size, data_size = ys.shape
    
    model = NeuralODE(data_size, width_size, depth, key=model_key)

    @eqx.filter_value_and_grad
    def grad_loss(model, ti, yi):
        y_pred = jax.vmap(model, in_axes=(None, 0))(ti, yi[:, 0])
        return jnp.mean((yi - y_pred) ** 2)

    @eqx.filter_jit
    def make_step(ti, yi, model, opt_state):
        loss, grads = grad_loss(model, ti, yi)
        updates, opt_state = optim.update(grads, opt_state)
        model = eqx.apply_updates(model, updates)
        return loss, model, opt_state

    for lr, steps, length in zip(lr_strategy, steps_strategy, length_strategy):
        optim = optax.adabelief(lr)
        opt_state = optim.init(eqx.filter(model, eqx.is_inexact_array))
        _ts = ts[: int(length_size * length)]
        _ys = ys[:, : int(length_size * length)]
        for step, (yi,) in zip(
            range(steps), dataloader((_ys,), batch_size, key=loader_key)
        ):
            start = time.time()
            loss, model, opt_state = make_step(_ts, yi, model, opt_state)
            end = time.time()

    return ts, ys, model, data_min, data_max

In [26]:
def RMSE_TR(ts, data, model, data_max, data_min):
    
    data_m = []
    data_o = []
    
    mae = []
    rmses = []
    cvrmse = []
    
    for i in range(data.shape[0]):
        
        model_y = model(ts, data[i, 0])
        
        original_data_o_i = data[i,:,1] * (data_max[i] - data_min[i]) + data_min[i]
        original_data_s_i = model_y * (data_max[i] - data_min[i]) + data_min[i]

        
        index = (np.linspace(0,999,25)).astype(int)
        
        data_m.append(original_data_s_i[index[0:23], 1])
        data_o.append(original_data_o_i[index[0:23]])
    
    data_m = np.concatenate(data_m)
    data_o = np.concatenate(data_o)

    
    for j in range(len(data_o) - 24):
    
        mae_value = np.mean(np.absolute(data_o[j:j+24] - data_m[j:j+24]))
        mae.append(mae_value)

        rmse_value = np.sqrt(np.mean((data_o[j:j+24] - data_m[j:j+24])**2))
        rmses.append(rmse_value)

        cvrmse_value = rmse_value / (np.mean(data_o[j:j+24]))
        cvrmse.append(cvrmse_value)

    return mae, np.mean(mae),rmses, np.mean(rmses), cvrmse, np.mean(cvrmse)

In [27]:
def RMSE_TE(ts, data, model, data_max_p, data_min_p, data_max,data_min):
    

    
    mae = []
    rmses = []
    cvrmse = []
    
    for i in range(0,data.shape[0] - 1):
        
        model_y = model(ts, data[i, 0])
        startpoint = model_y[999]
        startpoint = startpoint.at[0].set(0)
        model_y_next = model(ts, startpoint)

        
        original_data_o_i = data[i,:,1] * (data_max[i] - data_min[i]) + data_min[i]
        original_data_o_i_next = data[i+1,:,1] * (data_max[i+1] - data_min[i+1]) + data_min[i+1]
        original_data_s_i = model_y * (data_max_p[i] - data_min_p[i]) + data_min_p[i]
        original_data_s_i_next = model_y_next * (data_max_p[i+1] - data_min_p[i+1]) + data_min_p[i+1]



        
        index = (np.linspace(0,999,25)).astype(int)
        original_data_o_i = original_data_o_i[index]
        original_data_o_i_next = original_data_o_i_next[index]
        original_data_s_i = original_data_s_i[index]
        original_data_s_i_next = original_data_s_i_next[index]
        
        

        for j in range(24):
            
            data_o = np.concatenate([original_data_o_i[j:24],original_data_o_i_next[0:j]])
            data_s = np.concatenate([original_data_s_i[j:24,1],original_data_s_i_next[0:j,1]])
       
            mae_value = np.mean(np.absolute(data_o - data_s))
            mae.append(mae_value)

            rmse_value = np.sqrt(np.mean((data_o - data_s)**2))
            rmses.append(rmse_value)

            cvrmse_value = rmse_value / (np.mean(data_o))
            cvrmse.append(cvrmse_value)

    return mae, np.mean(mae),rmses, np.mean(rmses), cvrmse, np.mean(cvrmse)

In [32]:
def forecasting(number):

    energy = dataload(number)

    [data,data_min,data_max] = reshape_minmax(energy, number)

    [ts,ys] = combination(data, number)
    
    return ts,ys,data_max, data_min

In [33]:
def maxmin_r(number_of_day, data_max_f, data_min_f, selected_indices, remain_indices):
    data_max_pre = np.zeros(number_of_day)
    data_min_pre = np.zeros(number_of_day)
    
    
    data_max_pre[selected_indices] = data_max_f[selected_indices]
    data_min_pre[selected_indices] = data_min_f[selected_indices]

    
    for idx in remain_indices:
        
        if idx < 1:
            data_max_pre[idx] = data_max_f[idx]  
            data_min_pre[idx] = data_min_f[idx]
        else:
            data_max_pre[idx] = data_max_f[idx - 1]
            data_min_pre[idx] = data_min_f[idx - 1]
            

    return data_max_pre, data_min_pre

In [39]:
number_of_day = int(150)
[ts_f,ys_f,data_max_f,data_min_f] = forecasting(number_of_day)
RS = [1.1,1.2,1.3,1.4,1.6]
size = [75,65,55,45,35,25,15,5]
bs = [2,10,10,10,10,10,10,2]

In [ ]:
MAE_tr = []
MAE_te = []
RMSE_tr = []
RMSE_te = []
CV_tr = []
CV_te = []

for i in range(8):

    for j in range(5):
        
        [ts, ys, model, data_min, data_max] = main(number_of_day,size[i],(2000,5000),RS[j],batch_size = bs[i])
        [mae, mae_mean,rmse,rmse_mean,cvrmse,cvrmse_mean] = RMSE_TR(ts, ys, model, data_max, data_min)
        
        MAE_tr.append(mae_mean)
        RMSE_tr.append(rmse_mean)
        CV_tr.append(cvrmse_mean)
        
        random.seed(RS[j])
        selected_indices = random.sample(range(number_of_day), int(size[i]))
        remaining_indices = [i for i in range(number_of_day) if i not in selected_indices]

        [data_max_p,data_min_p] = maxmin_r(number_of_day,data_max_f,data_min_f, selected_indices,remaining_indices)

        [mae, mae_mean,rmse,rmse_mean,cvrmse,cvrmse_mean] = RMSE_TE(ts_f, ys_f, model, data_max_p, data_min_p, data_max_f, data_min_f)

        MAE_te.append(mae_mean)
        RMSE_te.append(rmse_mean)
        CV_te.append(cvrmse_mean)

data_overal = [MAE_tr, MAE_te, RMSE_tr, RMSE_te, CV_tr, CV_te]
column_names = ["MAE_tr", "MAE_te", "RMSE_tr", "RMSE_te", "CV_tr", "CV_te"]
data_df = pd.DataFrame(data_overal, column_names)
data_df.to_excel("xxx.xlsx", index=False)